In [1]:
#Notebook description

#This notebook is being used to evaluate the techinical market conditions of a single asset and assess
#the appropriate strategy to take in order to maximize returns.

In [2]:
#take all compuation functions and put them in a separate file

#simplify the date x axis on the percent drawdown chart

#default the zoom range to a comfortable range, and create a dropdown to select the time range for Volatility section

#properly label and annoate the garch models

#Remove the VIX charting, its redudnant now that we have the volatility models

In [3]:
#Load libraries 
import logging
logger = logging.getLogger('yfinance')
logger.disabled = True
logger.propagate = False
# Load libraries
from Quantapp.Plotter import Plotter
from Quantapp.Computation import Computation
from Quantapp.EconomicData import EconomicData

import numpy as np
import json
import os
import pandas as pd
import yfinance as yf
from statsmodels.tsa.stattools import coint
from IPython.display import display
from concurrent.futures import ThreadPoolExecutor
from plotly.subplots import make_subplots
from datetime import datetime
import statsmodels.api as sm
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as sp
import plotly.graph_objects as go
import plotly.graph_objects as go
import pandas as pd
from plotly.subplots import make_subplots

#shut down warnings
import warnings
warnings.filterwarnings("ignore")


qc = Computation()
qp = Plotter()
qe = EconomicData()

def simplify_datetime_index(series):
    """
    Simplifies the DateTime index of a Series to contain only the date (YYYY-MM-DD),
    maintaining it as a DateTimeIndex without timezone information.
    
    Parameters:
        series (pd.Series): The input Series with a DateTimeIndex.
    
    Returns:
        pd.Series: The Series with the DateTime index simplified to YYYY-MM-DD.
    """
    if not isinstance(series.index, pd.DatetimeIndex):
        raise TypeError("The Series index must be a DateTimeIndex.")
    
    # Remove timezone information if present
    if series.index.tz is not None:
        series = series.copy()
        series.index = series.index.tz_convert('UTC').tz_localize(None)
    
    # Normalize the index to remove the time component
    series.index = series.index.normalize()
    
    return series
                

In [4]:
#PARAMETERS

interval = '1d'
period     = '10y'
risk_free_rate = 0.02 / 252  # Annualized risk-free rate divided by trading days

time_frame_week = 7
# Define timeframe lists for different strategies
position_time_frames = [21, 50, 200]  # For position trading (longer term)
swing_time_frames = [3, 9, 21]        # For swing trading (shorter term)


#should be a string or None
ticker_str = 'XLC'  # The ticker symbol for the asset to analyze
primary_benchmark_str = 'SPY'
secondary_benchmark_str = None
length_of_plots = 10 # in years
trading_strategy = 'position' # Select which strategy to use (change to "swing" for swing trading)

# Populate the time frame variables based on selected strategy
if trading_strategy == "position":
    time_frame_short = position_time_frames[0]
    time_frame_mid = position_time_frames[1]
    time_frame_long = position_time_frames[2]
else:  # swing trading
    time_frame_short = swing_time_frames[0]
    time_frame_mid = swing_time_frames[1]
    time_frame_long = swing_time_frames[2]



In [5]:
#Load data

ticker = yf.Ticker(ticker_str).history(period=period, interval=interval)
vix = yf.Ticker('^VIX').history(period=period, interval=interval)

# Fetch and simplify benchmark data only if the strings are not None
if primary_benchmark_str is not None:
    primary_benchmark = yf.Ticker(primary_benchmark_str).history(period=period, interval=interval)
    primary_benchmark = simplify_datetime_index(primary_benchmark)
else:
    primary_benchmark = None

if secondary_benchmark_str is not None:
    secondary_benchmark = yf.Ticker(secondary_benchmark_str).history(period=period, interval=interval)
    secondary_benchmark = simplify_datetime_index(secondary_benchmark)
else:
    secondary_benchmark = None

# Simplify ticker and vix
ticker = simplify_datetime_index(ticker)
vix = simplify_datetime_index(vix)

# Align ticker data with primary benchmark if available
if primary_benchmark is not None:
    ticker = ticker[ticker.index.isin(primary_benchmark.index)]

# Calculate ticker returns
ticker_monthly_returns = qc.calculate_returns(ticker, frequency='monthly')
ticker_weekly_returns = qc.calculate_returns(ticker, frequency='weekly')
ticker_daily_returns = qc.calculate_returns(ticker, frequency='daily')

# Calculate primary benchmark returns if it exists
if primary_benchmark is not None:
    primary_benchmark_monthly_returns = qc.calculate_returns(primary_benchmark, frequency='monthly')
    primary_benchmark_weekly_returns = qc.calculate_returns(primary_benchmark, frequency='weekly')
    primary_benchmark_daily_returns = qc.calculate_returns(primary_benchmark, frequency='daily')

# Calculate secondary benchmark returns if it exists
if secondary_benchmark is not None:
    secondary_benchmark_monthly_returns = qc.calculate_returns(secondary_benchmark, frequency='monthly')
    secondary_benchmark_weekly_returns = qc.calculate_returns(secondary_benchmark, frequency='weekly')
    secondary_benchmark_daily_returns = qc.calculate_returns(secondary_benchmark, frequency='daily')

# Calculate VIX returns
vix_monthly_returns = qc.calculate_returns(vix, frequency='monthly')
vix_weekly_returns = qc.calculate_returns(vix, frequency='weekly')
vix_daily_returns = qc.calculate_returns(vix, frequency='daily')


In [6]:
#REGIME CHANGES: Candlestick with RSI and Bollinger Bands

def calculate_percentage_drop(ticker, n=14):
    """
    Calculate the percentage drop from the highest peak in a rolling window.

    Parameters:
    - ticker: pd.DataFrame with 'Close' prices indexed by date.
    - n: Number of days for the rolling window.

    Returns:
    - pd.DataFrame with 'Close', 'HighestHigh', and 'PercentageDrop' columns.
    """
    # Ensure ticker is a DataFrame and has a 'Close' column
    if 'Close' not in ticker.columns:
        raise ValueError("The DataFrame must contain a 'Close' column.")
    
    # Calculate the highest peak in the last n days
    ticker['HighestHigh'] = ticker['Close'].rolling(window=n, min_periods=1).max()
    
    # Calculate the percentage drop from the highest peak
    ticker['PercentageDrop'] = -((ticker['HighestHigh'] - ticker['Close']) / ticker['HighestHigh']) * 100
    
    return ticker

def plot_candlestick(ticker_data, drop_window=14, period='1Y', bollinger_window=21, title="Candlestick With Bollinger Bands"):
    """
    Plots the candlestick chart with Bollinger Bands for the given stock data.
    
    Parameters:
    - ticker_data: DataFrame containing candlestick data with 'Open', 'High', 'Low', 'Close' columns.
    - drop_window: Number of days for calculating the percentage drop.
    - period: Period to filter the data.
    - bollinger_window: Window for the moving average to calculate Bollinger Bands.
    - title: Title of the plot.
    """
    # Remove weekends/holidays and calculate percentage drop
    ticker_data = ticker_data[ticker_data.index.dayofweek < 5]
    holidays = pd.to_datetime(['2023-01-01', '2023-12-25'])  # Add more holidays as needed
    ticker_data = ticker_data[~ticker_data.index.isin(holidays)]
    ticker_data = calculate_percentage_drop(ticker_data, n=drop_window)
    mean_drop = ticker_data['PercentageDrop'].mean()
    std_drop = ticker_data['PercentageDrop'].std()
  
    # Filter data for the specified period
    period_data = ticker_data.last(period)

    # Define bar colors
    colors = [
        'red' if drop < mean_drop - 0.5 * std_drop
        else 'blue' if drop < mean_drop + 0.25 * std_drop
        else 'green'
        for drop in period_data['PercentageDrop']
    ]

    # Calculate Bollinger Bands
    ma = period_data['Close'].rolling(window=bollinger_window).mean()
    std = period_data['Close'].rolling(window=bollinger_window).std()

    bollinger_bands = {}
    for k in [1, 2, 3]:
        bollinger_bands[f'Upper_{k}'] = ma + (std * k)
        bollinger_bands[f'Lower_{k}'] = ma - (std * k)
    bollinger_df = pd.DataFrame(bollinger_bands)
    
    # Create a single-figure candlestick chart
    fig = go.Figure()

    # Add candlestick data
    for i, color in enumerate(colors):
        fig.add_trace(go.Candlestick(
            x=[period_data.index[i]],
            open=[period_data['Open'].iloc[i]],
            high=[period_data['High'].iloc[i]],
            low=[period_data['Low'].iloc[i]],
            close=[period_data['Close'].iloc[i]],
            increasing_line_color=color,
            decreasing_line_color=color,
            showlegend=False
        ))

    # Add Bollinger Bands
    for k in [1, 2, 3]:
        fig.add_trace(go.Scatter(
            x=period_data.index,
            y=bollinger_df[f'Upper_{k}'],
            mode='lines',
            line=dict(width=1, dash='dash'),
            name=f'Upper Band {k} SD'
        ))
        fig.add_trace(go.Scatter(
            x=period_data.index,
            y=bollinger_df[f'Lower_{k}'],
            mode='lines',
            line=dict(width=1, dash='dash'),
            name=f'Lower Band {k} SD'
        ))

    fig.update_layout(
        title=title,
        xaxis_title='Date',
        yaxis_title='Price',
        height=900,
        template='plotly_dark',
        yaxis=dict(autorange=True, fixedrange=False),
        xaxis=dict(
            rangeslider=dict(visible=False),
            tickangle=-45,
            showgrid=True,
            zeroline=False
        )
    )

    fig.show()

plot_candlestick(ticker, period='2y', bollinger_window=50, title="Candlestick with 50-Period Bollinger Bands")
plot_candlestick(ticker, period='5y', bollinger_window=200, title="Candlestick with 200-Period Bollinger Bands ")


In [7]:
#REGIME CHANGES: Percentage Drop from Highest Peak

# this code block visualizes the regime changes of the SPY ETF
# the regime changes tell us when the market is in a bull or bear market thus telling us the conditions
# that tell us when to buy or sell. 

# for example:
# if the market is in a bear market, we should be selling or shorting assets
# if the market is in a bull market, we should be buying or longing assets
# if the market is in a neutral market, we should be holding, not trading,or entering neutral positions

# the regime changes tell us to exclusively enter positions according to the market conditions and no morre
# it also tells us when to cut losses
n = int(252 / 2)

# Plot drawdowns for the main ticker
ticker_drawdowns = qc.calculate_percentage_drop(ticker)
qp.plot_percentage_drop(ticker_drawdowns, n=n, title=f'{ticker_str} Percentage Drop from Highest Peak')
'''
# Conditionally plot primary benchmark if available
if primary_benchmark is not None:
    primary_drawdowns = qc.calculate_percentage_drop(primary_benchmark)
    qp.plot_percentage_drop(primary_drawdowns, n=n, title=f'Benchmark 1: {primary_benchmark_str} Percentage Drop from Highest Peak')

# Conditionally plot secondary benchmark if available
if secondary_benchmark is not None:
    secondary_drawdowns = qc.calculate_percentage_drop(secondary_benchmark)
    qp.plot_percentage_drop(secondary_drawdowns, n=n, title=f'Benchmark 2: {secondary_benchmark_str} Percentage Drop from Highest Peak')
'''

"\n# Conditionally plot primary benchmark if available\nif primary_benchmark is not None:\n    primary_drawdowns = qc.calculate_percentage_drop(primary_benchmark)\n    qp.plot_percentage_drop(primary_drawdowns, n=n, title=f'Benchmark 1: {primary_benchmark_str} Percentage Drop from Highest Peak')\n\n# Conditionally plot secondary benchmark if available\nif secondary_benchmark is not None:\n    secondary_drawdowns = qc.calculate_percentage_drop(secondary_benchmark)\n    qp.plot_percentage_drop(secondary_drawdowns, n=n, title=f'Benchmark 2: {secondary_benchmark_str} Percentage Drop from Highest Peak')\n"

In [8]:
#Volatility: VIX FIX 

#create a function that takes in a series of data and returns the vix fix
# the vix fix is a measure of the volatility premium
# it was created by Larry Williams

def vix_fix(series, window=22):
    """
    Computes the VIX Fix (Larry Williams) for a given price series.

    Parameters:
    - series (pd.Series): The input time series of prices.
    - window (int): The number of periods to look back for the highest high.

    Returns:
    - pd.Series: VIX Fix values.
    """
    highest_close = series.rolling(window=window).max()
    vix_fix_values = 100 * (highest_close - series) / highest_close
    return vix_fix_values

def plot_series_with_stdev_bands(
    data_series,
    stdev_values=[-0.5, 0.5, 1.5, 3],
    title="Series with Mean & Standard Deviations"
):
    """
    Plots a given data series, adding horizontal lines for mean and multiple standard deviations.
    Shades the regions between standard deviation bands with distinct colors:
    - Between -0.5 and 0.5 standard deviations: Green
    - Between 0.5 and 1.5 standard deviations: Yellow
    - Between 1.5 and 3 standard deviations: Red

    Parameters:
    - data_series (pd.Series): Any precomputed series of values to plot.
    - stdev_values (list of float): Multipliers for standard deviations to plot (e.g., [-0.5, 0.5, 1.5, 3]).
    - title (str): Chart title.
    """
    fig = go.Figure()

    # Plot data_series
    fig.add_trace(go.Scatter(
        x=data_series.index,
        y=data_series,
        mode='lines',
        name='Data',
        line=dict(color='yellow')
    ))

    # Compute mean and std
    mean_val = data_series.mean()
    std_val = data_series.std()

    # Add horizontal line for mean
    fig.add_hline(
        y=mean_val,
        line_color="white",
        line_dash="dash",
        annotation_text=f"Mean: {mean_val:.2f}",
        annotation_position="bottom right"
    )

    # Add horizontal lines for each standard deviation
    for stdev in stdev_values:
        sd_line = mean_val + stdev * std_val
        fig.add_hline(
            y=sd_line,
            line_color="white",
            line_dash="dot",
            annotation_text=f"{stdev} SD: {sd_line:.2f}",
            annotation_position="bottom right"
        )

    # Define colors for each shading band
    shade_colors = [
        "rgba(0, 255, 0, 0.3)",    # Green for -0.5 to 0.5
        "rgba(255, 255, 0, 0.5)",  # Yellow for 0.5 to 1.5
        "rgba(255, 0, 0, 0.7)"     # Red for 1.5 to 3
    ]

    # Sort stdev_values for consistent shading
    stdev_values_sorted = sorted(stdev_values)

    # Shade regions between consecutive standard deviation bands
    for i in range(len(stdev_values_sorted) - 1):
        lower_stdev = stdev_values_sorted[i]
        upper_stdev = stdev_values_sorted[i + 1]
        y0 = mean_val + lower_stdev * std_val
        y1 = mean_val + upper_stdev * std_val
        color = shade_colors[i] if i < len(shade_colors) else "rgba(255, 0, 0, 0.7)"
        
        fig.add_shape(
            type="rect",
            xref="x",
            yref="y",
            x0=data_series.index.min(),
            y0=y0,
            x1=data_series.index.max(),
            y1=y1,
            fillcolor=color,
            layer="below",
            line_width=0
        )
        
    fig.update_layout(
        title=title,
        xaxis_title='Date',
        yaxis_title='Value',
        template='plotly_dark',
        height=1000,
        xaxis=dict(
            # Set range to latest year only
            range=[
                data_series.index[-1] - pd.DateOffset(years=1), 
                data_series.index[-1]
            ]
        ),
        # Set y-axis range based on data from latest year only
        yaxis=dict(
            range=[
                data_series.min(),  # Minimum value of the data series
                data_series.max()   # Maximum value of the data series
            ]
        ),
    )

    fig.show()

def plot_series_with_stdev_bands(
    data_series,
    stdev_values=[-0.5, 0.5, 1.5, 3],
    num_years=5,
    title="Series with Mean & Standard Deviations"
):
    """
    Plots a given data series, adding horizontal lines for mean and multiple standard deviations.
    Shades the regions between standard deviation bands with distinct colors:
    - Between -0.5 and 0.5 standard deviations: Green
    - Between 0.5 and 1.5 standard deviations: Yellow
    - Between 1.5 and 3 standard deviations: Red

    Parameters:
    - data_series (pd.Series): Any precomputed series of values to plot.
    - stdev_values (list of float): Multipliers for standard deviations to plot (e.g., [-0.5, 0.5, 1.5, 3]).
    - num_years (int): Number of years to zoom in on the chart.
    - title (str): Chart title.
    """
    # Filter data to the specified number of years
    zoom_start = data_series.index[-1] - pd.DateOffset(years=num_years)
    zoom_data = data_series.loc[data_series.index >= zoom_start]

    fig = go.Figure()

    # Plot data_series
    fig.add_trace(go.Scatter(
        x=data_series.index,
        y=data_series,
        mode='lines',
        name='Data',
        line=dict(color='yellow')
    ))

    # Compute mean and std
    mean_val = data_series.mean()
    std_val = data_series.std()

    # Add horizontal line for mean
    fig.add_hline(
        y=mean_val,
        line_color="white",
        line_dash="dash",
        annotation_text=f"Mean: {mean_val:.2f}",
        annotation_position="bottom right"
    )

    # Add horizontal lines for each standard deviation
    for stdev in stdev_values:
        sd_line = mean_val + stdev * std_val
        fig.add_hline(
            y=sd_line,
            line_color="white",
            line_dash="dot",
            annotation_text=f"{stdev} SD: {sd_line:.2f}",
            annotation_position="bottom right"
        )

    # Define colors for each shading band
    shade_colors = [
        "rgba(0, 255, 0, 0.3)",    # Green for -0.5 to 0.5
        "rgba(255, 255, 0, 0.5)",  # Yellow for 0.5 to 1.5
        "rgba(255, 0, 0, 0.7)"     # Red for 1.5 to 3
    ]

    # Sort stdev_values for consistent shading
    stdev_values_sorted = sorted(stdev_values)

    # Shade regions between consecutive standard deviation bands
    for i in range(len(stdev_values_sorted) - 1):
        lower_stdev = stdev_values_sorted[i]
        upper_stdev = stdev_values_sorted[i + 1]
        y0 = mean_val + lower_stdev * std_val
        y1 = mean_val + upper_stdev * std_val
        color = shade_colors[i] if i < len(shade_colors) else "rgba(255, 0, 0, 0.7)"

        fig.add_shape(
            type="rect",
            xref="x",
            yref="y",
            x0=data_series.index.min(),
            y0=y0,
            x1=data_series.index.max(),
            y1=y1,
            fillcolor=color,
            layer="below",
            line_width=0
        )

    # Update layout
    fig.update_layout(
        title=title,
        xaxis_title='Date',
        yaxis_title='Value',
        template='plotly_dark',
        height=800,
        xaxis=dict(
            # Set range to the latest num_years only
            range=[zoom_start, data_series.index[-1]]
        ),
        # Adjust y-axis range based on filtered data
        yaxis=dict(
            range=[zoom_data.min(), zoom_data.max()]
        ),
    )

    fig.show()   
# ...existing code...
ticker_vix_fix = vix_fix(ticker['Close'])
if primary_benchmark_str is not None:
    primary_benchmark_vix_fix = vix_fix(primary_benchmark['Close'])
if secondary_benchmark_str is not None:
    secondary_benchmark_vix_fix = vix_fix(secondary_benchmark['Close'])


plot_series_with_stdev_bands(
    ticker_vix_fix,
    title='VIX Fix with Mean and Standard Deviations',
    stdev_values=[-0.5, 0.5, 1.5, 3]
)
'''
if primary_benchmark_str is not None:
    primary_benchmark_vix_fix = vix_fix(primary_benchmark['Close'])
    plot_series_with_stdev_bands(
        primary_benchmark_vix_fix,
        title=f'Benchmark 1 ({primary_benchmark_str}) VIX Fix with Mean and Standard Deviations',
        stdev_values=[-0.5, 0.5, 1.5, 3]
    )

if secondary_benchmark_str is not None:
    secondary_benchmark_vix_fix = vix_fix(secondary_benchmark['Close'])
    plot_series_with_stdev_bands(
        secondary_benchmark_vix_fix,
        title=f'Benchmark 2 ({secondary_benchmark_str}) VIX Fix with Mean and Standard Deviations',
        stdev_values=[-0.5, 0.5, 1.5, 3]
    )'''

"\nif primary_benchmark_str is not None:\n    primary_benchmark_vix_fix = vix_fix(primary_benchmark['Close'])\n    plot_series_with_stdev_bands(\n        primary_benchmark_vix_fix,\n        title=f'Benchmark 1 ({primary_benchmark_str}) VIX Fix with Mean and Standard Deviations',\n        stdev_values=[-0.5, 0.5, 1.5, 3]\n    )\n\nif secondary_benchmark_str is not None:\n    secondary_benchmark_vix_fix = vix_fix(secondary_benchmark['Close'])\n    plot_series_with_stdev_bands(\n        secondary_benchmark_vix_fix,\n        title=f'Benchmark 2 ({secondary_benchmark_str}) VIX Fix with Mean and Standard Deviations',\n        stdev_values=[-0.5, 0.5, 1.5, 3]\n    )"

In [9]:
#SEASONALITY: Monthly Seasonality

fig_ticker_Seasonality_Monthly = qp.plot_seasonality(ticker_monthly_returns, f'Monthly Seasonality: {ticker_str}', 'monthly')
fig_ticker_Seasonality_Monthly.show()
'''
if primary_benchmark_str is not None:
    fig_primary_benchmark_Seasonality_Monthly = qp.plot_seasonality(primary_benchmark_monthly_returns, f'Monthly Seasonality: {primary_benchmark_str}', 'monthly')
    fig_primary_benchmark_Seasonality_Monthly.show()

if secondary_benchmark_str is not None:
    fig_secondary_benchmark_Seasonality_Monthly = qp.plot_seasonality(secondary_benchmark_monthly_returns, f'Monthly Seasonality: {secondary_benchmark_str}', 'monthly')
    fig_secondary_benchmark_Seasonality_Monthly.show()
'''

"\nif primary_benchmark_str is not None:\n    fig_primary_benchmark_Seasonality_Monthly = qp.plot_seasonality(primary_benchmark_monthly_returns, f'Monthly Seasonality: {primary_benchmark_str}', 'monthly')\n    fig_primary_benchmark_Seasonality_Monthly.show()\n\nif secondary_benchmark_str is not None:\n    fig_secondary_benchmark_Seasonality_Monthly = qp.plot_seasonality(secondary_benchmark_monthly_returns, f'Monthly Seasonality: {secondary_benchmark_str}', 'monthly')\n    fig_secondary_benchmark_Seasonality_Monthly.show()\n"

In [10]:
#SEASONALITY: Weekly Seasonality
fig_ticker_Seasonality_weekly = qp.plot_seasonality(ticker_weekly_returns, f'Weekly Seasonality: {ticker_str}', 'weekly')
fig_ticker_Seasonality_weekly.show()
'''
if primary_benchmark_str is not None:
    fig_primary_benchmark_Seasonality_weekly = qp.plot_seasonality(primary_benchmark_weekly_returns, f'Weekly Seasonality: {primary_benchmark_str}', 'weekly')
    fig_primary_benchmark_Seasonality_weekly.show()

if secondary_benchmark_str is not None:
    fig_secondary_benchmark_Seasonality_weekly = qp.plot_seasonality(secondary_benchmark_weekly_returns, f'Weekly Seasonality: {secondary_benchmark_str}', 'weekly')
    fig_secondary_benchmark_Seasonality_weekly.show()
    
'''

"\nif primary_benchmark_str is not None:\n    fig_primary_benchmark_Seasonality_weekly = qp.plot_seasonality(primary_benchmark_weekly_returns, f'Weekly Seasonality: {primary_benchmark_str}', 'weekly')\n    fig_primary_benchmark_Seasonality_weekly.show()\n\nif secondary_benchmark_str is not None:\n    fig_secondary_benchmark_Seasonality_weekly = qp.plot_seasonality(secondary_benchmark_weekly_returns, f'Weekly Seasonality: {secondary_benchmark_str}', 'weekly')\n    fig_secondary_benchmark_Seasonality_weekly.show()\n    \n"

In [11]:
#SEASONALITY: Daily Seasonality
fig_ticker_Seasonality_daily = qp.plot_seasonality(ticker_daily_returns, f'Daily Seasonality: {ticker_str}', 'daily')
fig_ticker_Seasonality_daily.show()
'''
if primary_benchmark_str is not None:
    fig_primary_benchmark_Seasonality_daily = qp.plot_seasonality(primary_benchmark_daily_returns, f'Daily Seasonality: {primary_benchmark_str}', 'daily')
    fig_primary_benchmark_Seasonality_daily.show()
    

if secondary_benchmark_str is not None:
    fig_secondary_benchmark_Seasonality_daily = qp.plot_seasonality(secondary_benchmark_daily_returns, f'Daily Seasonality: {secondary_benchmark_str}', 'daily')
    fig_secondary_benchmark_Seasonality_daily.show()
'''

"\nif primary_benchmark_str is not None:\n    fig_primary_benchmark_Seasonality_daily = qp.plot_seasonality(primary_benchmark_daily_returns, f'Daily Seasonality: {primary_benchmark_str}', 'daily')\n    fig_primary_benchmark_Seasonality_daily.show()\n    \n\nif secondary_benchmark_str is not None:\n    fig_secondary_benchmark_Seasonality_daily = qp.plot_seasonality(secondary_benchmark_daily_returns, f'Daily Seasonality: {secondary_benchmark_str}', 'daily')\n    fig_secondary_benchmark_Seasonality_daily.show()\n"

In [12]:
#Calculations

sharpe_ticker_short = qc.calculate_risk_adjusted_returns(ticker['Close'], ratio_type='sharpe', windows=[time_frame_short])[f'sharpe_ratio_{time_frame_short}']
sharpe_ticker_mid = qc.calculate_risk_adjusted_returns(ticker['Close'], ratio_type='sharpe', windows=[time_frame_mid])[f'sharpe_ratio_{time_frame_mid}']
sharpe_ticker_long = qc.calculate_risk_adjusted_returns(ticker['Close'], ratio_type='sharpe', windows=[time_frame_long])[f'sharpe_ratio_{time_frame_long}']


if primary_benchmark is not None:
    spread_short_primary = primary_benchmark['Close'].pct_change(time_frame_short) - ticker['Close'].pct_change(time_frame_short)
    spread_mid_primary = primary_benchmark['Close'].pct_change(time_frame_mid) - ticker['Close'].pct_change(time_frame_mid)
    spread_long_primary = primary_benchmark['Close'].pct_change(time_frame_long) - ticker['Close'].pct_change(time_frame_long)
    
    sharpe_benchmark_short_primary = qc.calculate_risk_adjusted_returns(
        primary_benchmark['Close'], 
        ratio_type='sharpe', 
        windows=[time_frame_short]
    )[f'sharpe_ratio_{time_frame_short}']
    
    sharpe_benchmark_mid_primary = qc.calculate_risk_adjusted_returns(
        primary_benchmark['Close'], 
        ratio_type='sharpe', 
        windows=[time_frame_mid]
    )[f'sharpe_ratio_{time_frame_mid}']
    
    sharpe_benchmark_long_primary = qc.calculate_risk_adjusted_returns(
        primary_benchmark['Close'], 
        ratio_type='sharpe', 
        windows=[time_frame_long]
    )[f'sharpe_ratio_{time_frame_long}']
    
    sharpe_spread_short_primary = sharpe_benchmark_short_primary - sharpe_ticker_short
    sharpe_spread_mid_primary = sharpe_benchmark_mid_primary - sharpe_ticker_mid
    sharpe_spread_long_primary = sharpe_benchmark_long_primary - sharpe_ticker_long

# Calculate spreads and Sharpe ratios for the secondary benchmark if it exists
if secondary_benchmark is not None:
    spread_short_secondary = secondary_benchmark['Close'].pct_change(time_frame_short) - ticker['Close'].pct_change(time_frame_short)
    spread_mid_secondary = secondary_benchmark['Close'].pct_change(time_frame_mid) - ticker['Close'].pct_change(time_frame_mid)
    spread_long_secondary = secondary_benchmark['Close'].pct_change(time_frame_long) - ticker['Close'].pct_change(time_frame_long)
    
    sharpe_benchmark_short_secondary = qc.calculate_risk_adjusted_returns(
        secondary_benchmark['Close'], 
        ratio_type='sharpe', 
        windows=[time_frame_short]
    )[f'sharpe_ratio_{time_frame_short}']
    
    sharpe_benchmark_mid_secondary = qc.calculate_risk_adjusted_returns(
        secondary_benchmark['Close'], 
        ratio_type='sharpe', 
        windows=[time_frame_mid]
    )[f'sharpe_ratio_{time_frame_mid}']
    
    sharpe_benchmark_long_secondary = qc.calculate_risk_adjusted_returns(
        secondary_benchmark['Close'], 
        ratio_type='sharpe', 
        windows=[time_frame_long]
    )[f'sharpe_ratio_{time_frame_long}']
    
    sharpe_spread_short_secondary = sharpe_benchmark_short_secondary - sharpe_ticker_short
    sharpe_spread_mid_secondary = sharpe_benchmark_mid_secondary - sharpe_ticker_mid
    sharpe_spread_long_secondary = sharpe_benchmark_long_secondary - sharpe_ticker_long



In [13]:
#RISK ADJUSTED RETURNS: Short Term
# Plot for primary benchmark if it exists
'''
if primary_benchmark is not None:

    qp.create_spread_plot(
        sharpe_spread_short_primary,
        title=f'{time_frame_short} Day: Benchmark ({primary_benchmark_str}) minus {ticker_str} (Sharpe)',
        default_years=10
    ).show()
    
    qp.plot_returns(
        ticker['Close'],
        benchmark_series=primary_benchmark['Close'],
        plot_type='risk_adjusted',
        title=f'{ticker_str} Returns {time_frame_short} days',
        window=time_frame_short
    ).show()

# Plot for secondary benchmark if it exists
if secondary_benchmark is not None:

    qp.create_spread_plot(
        sharpe_spread_short_secondary,
        title=f'{time_frame_short} Day: Benchmark ({secondary_benchmark_str}) minus {ticker_str} (Sharpe)',
        default_years=10
    ).show()
    
    qp.plot_returns(
        ticker['Close'],
        benchmark_series=secondary_benchmark['Close'],
        plot_type='risk_adjusted',
        title=f'{ticker_str} Returns {time_frame_short} days',
        window=time_frame_short
    ).show()
  '''

"\nif primary_benchmark is not None:\n\n    qp.create_spread_plot(\n        sharpe_spread_short_primary,\n        title=f'{time_frame_short} Day: Benchmark ({primary_benchmark_str}) minus {ticker_str} (Sharpe)',\n        default_years=10\n    ).show()\n    \n    qp.plot_returns(\n        ticker['Close'],\n        benchmark_series=primary_benchmark['Close'],\n        plot_type='risk_adjusted',\n        title=f'{ticker_str} Returns {time_frame_short} days',\n        window=time_frame_short\n    ).show()\n\n# Plot for secondary benchmark if it exists\nif secondary_benchmark is not None:\n\n    qp.create_spread_plot(\n        sharpe_spread_short_secondary,\n        title=f'{time_frame_short} Day: Benchmark ({secondary_benchmark_str}) minus {ticker_str} (Sharpe)',\n        default_years=10\n    ).show()\n    \n    qp.plot_returns(\n        ticker['Close'],\n        benchmark_series=secondary_benchmark['Close'],\n        plot_type='risk_adjusted',\n        title=f'{ticker_str} Returns {time_

In [14]:
#RISK ADJUSTED RETURNS: Short
# Plot for primary benchmark if it exists

if primary_benchmark is not None:
    '''    
    qp.create_spread_plot(
        spread_mid_primary,
        title=f'{time_frame_short} Day: Benchmark ({primary_benchmark_str}) minus {ticker_str}',
        default_years=5
    ).show()
    '''
    qp.create_spread_plot(
        sharpe_spread_short_primary,
        title=f'{time_frame_short} Day: Benchmark ({primary_benchmark_str}) minus {ticker_str} (Sharpe)',
        default_years=length_of_plots
    ).show()
    
    qp.plot_returns(
        ticker['Close'],
        benchmark_series=primary_benchmark['Close'],
        plot_type='risk_adjusted',
        title=f'{ticker_str} Returns {time_frame_short} days',
        window=time_frame_short,
    default_years=length_of_plots
    ).show()

# Plot for secondary benchmark if it exists
if secondary_benchmark is not None:
    '''    
    qp.create_spread_plot(
        spread_mid_secondary,
        title=f'{time_frame_short} Day: Benchmark ({secondary_benchmark_str}) minus {ticker_str}',
        default_years=5
    ).show()
    '''
    qp.create_spread_plot(
        sharpe_spread_short_secondary,
        title=f'{time_frame_short} Day: Benchmark ({secondary_benchmark_str}) minus {ticker_str} (Sharpe)',
        default_years=length_of_plots
    ).show()
    
    qp.plot_returns(
        ticker['Close'],
        benchmark_series=secondary_benchmark['Close'],
        plot_type='risk_adjusted',
        title=f'{ticker_str} Returns {time_frame_short} days',
        window=time_frame_short,
        default_years=length_of_plots
    ).show()
  

In [15]:
#RISK ADJUSTED RETURNS: Mid Term
# Plot for primary benchmark if it exists

if primary_benchmark is not None:
    '''    
    qp.create_spread_plot(
        spread_mid_primary,
        title=f'{time_frame_mid} Day: Benchmark ({primary_benchmark_str}) minus {ticker_str}',
        default_years=5
    ).show()
    '''
    qp.create_spread_plot(
        sharpe_spread_mid_primary,
        title=f'{time_frame_mid} Day: Benchmark ({primary_benchmark_str}) minus {ticker_str} (Sharpe)',
        default_years=length_of_plots
    ).show()
    
    qp.plot_returns(
        ticker['Close'],
        benchmark_series=primary_benchmark['Close'],
        plot_type='risk_adjusted',
        title=f'{ticker_str} Returns {time_frame_mid} days',
        window=time_frame_mid,
    default_years=length_of_plots
    ).show()

# Plot for secondary benchmark if it exists
if secondary_benchmark is not None:
    '''    
    qp.create_spread_plot(
        spread_mid_secondary,
        title=f'{time_frame_mid} Day: Benchmark ({secondary_benchmark_str}) minus {ticker_str}',
        default_years=5
    ).show()
    '''
    qp.create_spread_plot(
        sharpe_spread_mid_secondary,
        title=f'{time_frame_mid} Day: Benchmark ({secondary_benchmark_str}) minus {ticker_str} (Sharpe)',
        default_years=length_of_plots
    ).show()
    
    qp.plot_returns(
        ticker['Close'],
        benchmark_series=secondary_benchmark['Close'],
        plot_type='risk_adjusted',
        title=f'{ticker_str} Returns {time_frame_mid} days',
        window=time_frame_mid,
        default_years=length_of_plots
    ).show()
  

In [16]:
#RISK ADJUSTED RETURNS: Long Term

if primary_benchmark is not None:
    '''    
    qp.create_spread_plot(
        spread_long_primary,
        title=f'{time_frame_long} Day: Benchmark ({primary_benchmark_str}) minus {ticker_str}',
        default_years=5
    ).show()
    '''
    qp.create_spread_plot(
        sharpe_spread_long_primary,
        title=f'{time_frame_long} Day: Benchmark ({primary_benchmark_str}) minus {ticker_str} (Sharpe)',
        default_years=length_of_plots
    ).show()
    
    qp.plot_returns(
        ticker['Close'],
        benchmark_series=primary_benchmark['Close'],
        plot_type='risk_adjusted',
        title=f'{ticker_str} Returns {time_frame_long} days',
        window=time_frame_long,
        default_years=length_of_plots
    ).show()

# Plot for secondary benchmark if it exists
if secondary_benchmark is not None:
    '''   
    qp.create_spread_plot(
        spread_long_secondary,
        title=f'{time_frame_long} Day: Benchmark ({secondary_benchmark_str}) minus {ticker_str}',
        default_years=5
    ).show()
    '''
    qp.create_spread_plot(
        sharpe_spread_long_secondary,
        title=f'{time_frame_long} Day: Benchmark ({secondary_benchmark_str}) minus {ticker_str} (Sharpe)',
        default_years=length_of_plots
    ).show()
    
    qp.plot_returns(
        ticker['Close'],
        benchmark_series=secondary_benchmark['Close'],
        plot_type='risk_adjusted',
        title=f'{ticker_str} Returns {time_frame_long} days',
        window=time_frame_long,
        default_years=length_of_plots
    ).show()
    

In [ ]:
#Fama-French Factor Analysis: Calculations
import yfinance as yf
import pandas as pd
import statsmodels.api as sm
# Define period & interval
period = "10y"
interval = "1d"

# Download Fama-French factors and asset price data

tickers = ["SPY", "SIZE", "VLUE", "QUAL", "USMV", "MTUM", "BIL"]
if primary_benchmark_str is not None:
    tickers.append(primary_benchmark_str)
if secondary_benchmark_str is not None:
    tickers.append(secondary_benchmark_str)
    
prices = {ticker: yf.Ticker(ticker).history(period=period, interval=interval)['Close'] for ticker in tickers}
prices_df = pd.DataFrame(prices)

# Calculate daily returns
returns = prices_df.pct_change().dropna()

# Calculate excess market return (Market - Risk-Free)
factor_returns = pd.DataFrame()
factor_returns['Mkt-RF'] = returns["SPY"] - returns["BIL"]
factor_returns['SMB'] = (returns["SIZE"] - returns["SPY"]) / 2
factor_returns['HML'] = (returns["VLUE"] - returns["QUAL"]) / 2
factor_returns['RMW'] = (returns["QUAL"] - returns["VLUE"]) / 2
factor_returns['CMA'] = (returns["USMV"] - returns["MTUM"]) / 2
#add the primary benchmark as a factor

if primary_benchmark_str is not None and primary_benchmark_str != 'SPY':
    factor_returns[primary_benchmark_str] = returns[primary_benchmark_str] - returns["BIL"]

if secondary_benchmark_str is not None and secondary_benchmark_str != 'SPY':
    factor_returns[secondary_benchmark_str] = returns[secondary_benchmark_str] - returns["BIL"]
    

factor_returns_capm = factor_returns[['Mkt-RF']].copy()
factor_returns_ff3 = factor_returns[['Mkt-RF', 'SMB', 'HML']].copy()
factor_returns_ff5 = factor_returns[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']].copy()
#primary_benchmark_str might exist, secondary_benchmark_str might exist, make sure to add them as factors
custom_list = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']

if primary_benchmark_str is not None and primary_benchmark_str != 'SPY':
    custom_list.append(primary_benchmark_str)
if secondary_benchmark_str is not None and secondary_benchmark_str != 'SPY':
    custom_list.append(secondary_benchmark_str)
    
factor_returns_ff5_custom = factor_returns[custom_list].copy()

stock_returns = yf.Ticker(ticker_str).history(period=period, interval=interval)['Close'].pct_change().dropna()

display(stock_returns)
display()

def rolling_regression(stock_returns, returns, factor_returns, window):
    """
    Computes a rolling OLS regression on an asset's excess returns relative to the risk-free rate
    using provided factor returns over a specified rolling window.
    
    For each rolling window, it calculates:
      - alpha (intercept)
      - beta for each factor
      - r_squared
      - adjusted r_squared
      
    Excess returns = stock_returns - returns["BIL"]

    Parameters:
        stock_returns (pd.Series): Series of asset returns.
        returns (pd.DataFrame): DataFrame containing returns for various tickers.
                               Must include the risk-free rate under the column "BIL".
        factor_returns (pd.DataFrame): DataFrame containing factor returns.
        window (int): The number of periods in each rolling window.

    Returns:
        pd.DataFrame: A DataFrame indexed by the end date of each window with columns:
                      "alpha", "<factor>_beta" for each factor, "r_squared", "adj_r_squared".
    """
    import statsmodels.api as sm
    import pandas as pd

    results = []
    # Loop over rolling window periods
    for end in range(window, len(stock_returns) + 1):
        # Define the window of dates for the current regression
        window_index = stock_returns.index[end - window:end]
        
        # Extract the window data
        window_stock_returns = stock_returns.loc[window_index]
        window_rf = returns["BIL"].loc[window_index]
    
        window_excess = window_stock_returns - window_rf
        window_factors = factor_returns.loc[window_index]
        
        # Prepare independent variables with a constant
        X = sm.add_constant(window_factors)
        y = window_excess
        
        # Run the OLS regression
        model = sm.OLS(y, X).fit()
        
        # Extract regression parameters
        regression_result = {"date": window_index[-1],
                             "alpha": model.params["const"],
                             "r_squared": model.rsquared,
                             "adj_r_squared": model.rsquared_adj}
        for factor in window_factors.columns:
            regression_result[f"{factor}_beta"] = model.params[factor]
        
        results.append(regression_result)
    
    # Create a DataFrame from the list of results and set the index to the window end dates
    rolling_results_df = pd.DataFrame(results)
    rolling_results_df.set_index("date", inplace=True)
    
    return rolling_results_df

rolling_results_ff5_custom           = rolling_regression(stock_returns, returns, factor_returns_ff5_custom, 252)

#rolling_results_ff5_custom


Date
2018-06-20 00:00:00-04:00    0.012410
2018-06-21 00:00:00-04:00   -0.006129
2018-06-22 00:00:00-04:00    0.004376
2018-06-25 00:00:00-04:00   -0.020598
2018-06-26 00:00:00-04:00    0.001658
Name: Close, dtype: float64

In [18]:
#Plotting the Fama-French Factor Analysis Results
from plotly.subplots import make_subplots

def plot_rolling_regression(rolling_results, ticker_str, factor_returns):
    # Plot the alpha
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=rolling_results.index,
        y=rolling_results['alpha'],
        mode='lines',
        name='Alpha'
    ))
    # Add a horizontal line at 0
    fig.add_shape(
        type="line",
        x0=rolling_results.index[0],
        y0=0,
        x1=rolling_results.index[-1],
        y1=0,
        line=dict(
            color="white",
            width=1,
            dash="dash"
        )
    )
    
    # Add a horizontal line for the mean of alpha
    mean_alpha = rolling_results['alpha'].mean()
    std_alpha = rolling_results['alpha'].std()
    fig.add_hline(
        y=mean_alpha,
        line_color="white",
        line_dash="dot",
        annotation_text=f"Mean: {mean_alpha:.2f}",
        annotation_position="bottom right"
    )
    
    # Add horizontal lines for ±1, ±1.5, ±2, ±3 standard deviations from the mean
    for i in [1, 1.5, 2, 3]:
        fig.add_hline(
            y=mean_alpha + i * std_alpha,
            line_color="red",
            line_dash="dash",
            annotation_text=f"+{i}σ: {mean_alpha + i * std_alpha:.2f}",
            annotation_position="top right"
        )
        fig.add_hline(
            y=mean_alpha - i * std_alpha,
            line_color="green",
            line_dash="dash",
            annotation_text=f"-{i}σ: {mean_alpha - i * std_alpha:.2f}",
            annotation_position="bottom right"
        )
    
    fig.update_layout(
        title=f'{ticker_str} Rolling Alpha',
        xaxis_title='Date',
        yaxis_title='Alpha',
        template='plotly_dark',
        height=600,
        xaxis=dict(
            rangeslider=dict(visible=False),
            tickangle=-45,
            showgrid=True,
            zeroline=False
        )
    )
    fig.show()
    # Create subplots: one row per factor
    # Exclude the 'RF' column so that we only plot factor betas (e.g., Mkt-RF, SMB, HML)
    factors_to_plot = [factor for factor in factor_returns.columns if factor != "RF"]
    num_factors = len(factors_to_plot)

    # Create subplots for each factor beta
    fig = make_subplots(
        rows=num_factors,
        cols=1,
        shared_xaxes=True,
        subplot_titles=[f"{factor} Beta" for factor in factors_to_plot]
    )

    # Add each beta trace along with its horizontal baseline:
    for i, factor in enumerate(factors_to_plot, start=1):
        fig.add_trace(
            go.Scatter(
                x=rolling_results.index,
                y=rolling_results[f'{factor}_beta'],
                mode='lines',
                name=f'{factor} Beta'
            ),
            row=i,
            col=1
        )
        baseline = 1 if factor == "Mkt-RF" else 0
        fig.add_hline(
            y=baseline,
            row=i,
            col=1,
            line=dict(color="white", dash="dash"),
            annotation_text=f"Baseline: {baseline}",
            annotation_position="bottom right"
        )

    fig.update_layout(
        title=f'{ticker_str} Rolling Betas',
        template='plotly_dark',
        height=400 * num_factors,
        showlegend=False
    )
    fig.show()
    # Plot the R-squared and adjusted R-squared
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=rolling_results.index,
        y=rolling_results['r_squared'],
        mode='lines',
        name='R-Squared'
    ))
    fig.add_trace(go.Scatter(
        x=rolling_results.index,
        y=rolling_results['adj_r_squared'],
        mode='lines',
        name='Adjusted R-Squared'
    ))
    
    #add a horizontal line at mean of the r-squared
    mean_r_squared = rolling_results['r_squared'].mean()
    fig.add_hline(
        y=mean_r_squared,
        line_color="white",
        line_dash="dot",
        annotation_text=f"Mean: {mean_r_squared:.2f}",
        annotation_position="bottom right"
    )
    
    #add a horizontal line at 0
    fig.add_shape(
        type="line",
        x0=rolling_results.index[0],
        y0=0,
        x1=rolling_results.index[-1],
        y1=0,
        line=dict(
            color="white",
            width=1,
            dash="dash"
        )
    )
    
    fig.update_layout(
        title=f'{ticker_str} Rolling R-Squared',
        xaxis_title='Date',
        yaxis_title='R-Squared',
        template='plotly_dark',
        height=600,
        xaxis=dict(
            rangeslider=dict(visible=False),
            tickangle=-45,
            showgrid=True,
            zeroline=False
        )
    )
    fig.show()

plot_rolling_regression(rolling_results_ff5_custom, ticker_str, factor_returns_ff5_custom)